In [1]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 3080


In [14]:
import os 
import argparse
import warnings
from tqdm import tqdm
from typing import Tuple, Dict, Optional

import torch
from torch.utils.data import DataLoader
from transformers import PreTrainedTokenizer

from llm import LLM
from utils import *
from prompt_dataset import PromptDataset


os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")
warnings.filterwarnings('ignore')
SEED=10

info = {
    "data_path": r"C:\Users\franc\Documents\Bridge_the_GAP\data\test_dataset.json",
    "contriever_search_results_path": r"C:\Users\franc\Documents\Bridge_the_GAP\data\processed\contriever_test_search_results_at150.pkl",
}

class DotDict:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

def parse_arguments(custom_args=None):
    """
    Mimics argparse to parse arguments for LLM generation. Accepts custom arguments as a dictionary for notebooks.
    """
    # Define default values
    default_args = {
        'output_dir': r'C:\Users\franc\Documents\Bridge_the_GAP\data\gen_res_example_llm',
        'llm_id': 'google/gemma-2-2b-it',
        'model_max_length': 4096,
        'gold_position': None,
        'num_documents_in_context': None,
        'get_documents_without_answer': True,
        'max_new_tokens': 50,
        'batch_size': None,
        'save_every': 250,
    }

    # If custom_args is provided, update defaults
    if custom_args:
        default_args.update(custom_args)

    # Perform validation
    if default_args['num_documents_in_context'] is None:
        raise ValueError("'num_documents_in_context' must be specified.")
    if default_args['num_documents_in_context'] <= 0:
        raise ValueError("'num_documents_in_context' must be a positive integer.")
    if default_args['gold_position'] is not None:
        if (default_args['gold_position'] < 0 or 
            default_args['gold_position'] >= default_args['num_documents_in_context']):
            raise ValueError("'gold_position' must be within the range of 'num_documents_in_context'.")

    return DotDict(**default_args)


def load_corpus(
    args: argparse.Namespace
) -> Tuple[List[Dict], Optional[Dict[int, int]]]:
    
    # Corpus with documents from Contriever
    corpus, full_to_subset_idx_map = read_test_corpus_with_random_and_contriever()

    return corpus, full_to_subset_idx_map


def load_search_results(args: argparse.Namespace) -> List[Tuple[List[int], List[float]]]:
    # Search results from Contriever
    search_results_path = info['contriever_search_results_path'] 

    search_results = read_pickle(search_results_path)
    return search_results


def initialize_dataset_and_loader(
    args: argparse.Namespace, 
    corpus: List[Dict], 
    full_to_subset_idx_map: Optional[Dict[int, int]], 
    search_results: List[Tuple[List[int], List[float]]], 
    tokenizer: PreTrainedTokenizer
) -> DataLoader:
    
    prompt_ds = PromptDataset(
        corpus=corpus, data_path=info['data_path'], 
        tokenizer=tokenizer, 
        max_tokenized_length=args.model_max_length - 2, 
        search_results=search_results,
        full_to_subset_idx_map=full_to_subset_idx_map,
        do_normalize_query=True, 
        num_documents_in_context=args.num_documents_in_context,
        gold_position=args.gold_position,
        get_documents_without_answer=args.get_documents_without_answer,
    )
    prompt_dataloader = DataLoader(
        prompt_ds,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=8,
        pin_memory=True,
    )
    return prompt_dataloader


def print_info(args: argparse.Namespace):
    print("INFO:")
    print(f"DATA: {info['data_path']}")
    print(f"MODEL: {args.llm_id}")
    print(f"GOLD POSITION: {args.gold_position}")
    print(f"NUM DOCUMENTS IN CONTEXT: {args.num_documents_in_context}")
    print(f"DOCUMENTS WITHOUT ANSWER: {args.get_documents_without_answer}")
    print(f"MAX NEW TOKENS: {args.max_new_tokens}")
    print(f"BATCH SIZE: {args.batch_size}")
    print(f"SAVE EVERY: {args.save_every}")


def generate_and_save(
    args: argparse.Namespace, 
    llm: LLM, 
    prompt_dataloader: DataLoader
):
    # Info from arguments
    llm_id = args.llm_id
    num_doc = args.num_documents_in_context
    save_every = args.save_every
    gold_pos = args.gold_position
    retriever_str = "contriever"
    answerless_str = "_answerless" if args.get_documents_without_answer else ""

    # Create the saving directory
    llm_folder = llm_id.split("/")[1] if '/' in llm_id else llm_id
    saving_dir = f"{args.output_dir}/{llm_folder}/test/classic/{retriever_str}/{num_doc}_doc"
    if not os.path.exists(saving_dir):
        os.makedirs(saving_dir)

    
    # MPT has a different answer string in the prompt
    answer_string_in_prompt = "### Response:" if 'mpt' in llm_id else "Answer:"

    all_info = []  
    for idx, prompt_batch in enumerate(tqdm(prompt_dataloader)):
        prompts = prompt_batch['prompt']
        generated_output = llm.generate(prompts, max_new_tokens=args.max_new_tokens)
        
        generated_answers = []
        for output in generated_output:
            start = output.find(answer_string_in_prompt) + len(answer_string_in_prompt)
            response = output[start:].strip()
            generated_answers.append(response)

        prompt_batch['generated_answer'] = generated_answers
        all_info.append(prompt_batch)
        
        if (idx + 1) % save_every == 0 or (idx + 1) == len(prompt_dataloader):
            print(f"Saving at {idx + 1}...")
            file_name = f"{saving_dir}/numdoc{num_doc}_gold_at{gold_pos}{answerless_str}_info_{idx+1}.pkl"
            write_pickle(all_info, file_name)
            all_info = []


def main():
    args = parse_arguments({
        'output_dir': r'C:\Users\franc\Documents\Bridge_the_GAP\data\gen_res_example_llm',
        'llm_id': 'google/gemma-2-2b-it',
        'model_max_length': 4096,
        'gold_position': None,
        'num_documents_in_context': 5,
        'get_documents_without_answer': True,
        'max_new_tokens': 50,
        'batch_size': None,
        'save_every': 250,
    })

    print("Loading LLM...")
    llm_id = args.llm_id
    llm = LLM(
        llm_id, device, quantization_bits=4, 
        model_max_length=args.model_max_length
    )
    tokenizer = llm.tokenizer
    print("LLM loaded")


    print("Loading corpus and search results...")
    corpus, full_to_subset_idx_map = load_corpus(args)
    search_results = load_search_results(args)
    print("Corpus and search results loaded")


    print("Loading prompt dataset...")
    prompt_dataloader = initialize_dataset_and_loader(
        args, corpus, full_to_subset_idx_map, search_results, tokenizer
    )
    print("Prompt dataset loaded")

    print_info(args)
    generate_and_save(args, llm, prompt_dataloader)



if __name__ == "__main__":
    seed_everything(SEED)
    main()

Loading LLM...


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.50s/it]


LLM loaded
Loading corpus and search results...
Corpus and search results loaded
Loading prompt dataset...


TypeError: PromptDataset.__init__() got an unexpected keyword argument 'get_documents_without_answer'

In [ ]:
# Show the results of generate_answer_llm
from utils import *

result_path=r"C:\Users\franc\Documents\Bridge_the_GAP\data\gen_res_example_llm\gemma-2-2b-it\test\classic\contriever\5_doc\numdoc5_gold_atNone_answerless_info_250.pkl"
data=read_pickle(result_path)

data

In [8]:
import os
import re 
import argparse
import warnings
from tqdm import tqdm
from typing import Tuple, Dict, Optional

import torch
from torch.utils.data import DataLoader
from transformers import PreTrainedTokenizer

from utils import *
from llm import LLM
from default_prompts import *
from prompt_dataset import PromptDataset


os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")
warnings.filterwarnings('ignore')
SEED=10

info = {
    "nq": {
        "test": {
            "data_path": r'C:\Users\franc\Documents\Bridge_the_GAP\data\test_dataset.json',
            "contriever_search_results_path": r"C:\Users\franc\Documents\Bridge_the_GAP\data\processed\contriever_test_search_results_at150.pkl",
        }
    },
}

class DotDict:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

def parse_arguments(custom_args=None):
    """
    Mimics argparse to parse arguments for LLM generation. Accepts custom arguments as a dictionary for notebooks.
    """
    # Define default values
    default_args = {
        'output_dir': r'C:\Users\franc\Documents\Bridge_the_GAP\data\gen_res_example_llm',
        'llm_id': 'google/gemma-2-2b-it',
        'dataset': 'nq',
        'model_max_length': 4096,
        'quantization_bits': 4,
        'use_model_chat_template': True, 
        'gold_position': None,
        'num_retrieved_documents': 5,
        'use_test': True,
        'padding_strategy': 'longest',
        'max_new_tokens': 50,
        'use_task_with_proof': False,
        'batch_size': None,
        'save_every': 250,
    }

    # If custom_args is provided, update defaults
    if custom_args:
        default_args.update(custom_args)

    # Perform validation
    if default_args['num_retrieved_documents'] is None:
        raise ValueError("'num_retrieved_documents' must be specified.")
    if default_args['num_retrieved_documents'] <= 0:
        raise ValueError("'num_retrieved_documents' must be a positive integer.")
    if default_args['gold_position'] is not None:
        if (default_args['gold_position'] < 0 or 
            default_args['gold_position'] >= default_args['num_retrieved_documents']):
            raise ValueError("'gold_position' must be within the range of 'num_retrieved_documents'.")

    return DotDict(**default_args)


def load_corpus(
    args: argparse.Namespace
) -> Tuple[List[Dict], Optional[Dict[int, int]]]:
    
    # Corpus with documents from Contriever
    corpus, full_to_subset_idx_map = read_test_corpus_with_random_and_contriever()

    return corpus, full_to_subset_idx_map

def load_search_results(args: argparse.Namespace) -> List[Tuple[List[int], List[float]]]:
    # Search results from Contriever
    search_results_path = info['nq']['test']['contriever_search_results_path'] 

    search_results = read_pickle(search_results_path)
    return search_results


def get_prompt_template(args: argparse.Namespace):
    prompt_configuration = args.dataset

    if args.use_model_chat_template:
        chat_task_template_str = chat_task_templates[args.llm_id]['template']
        
        task_instruction = task_instructions[prompt_configuration]
        if args.use_task_with_proof:
            task_instruction = task_instructions['qa_proof'][prompt_configuration]

        prompt_template = apply_chat_task_template(chat_task_template_str, task_instruction)
    else:
        task_template = task_templates[prompt_configuration]

        if args.use_task_with_proof:
            task_template = task_templates['qa_proof'][prompt_configuration]

        prompt_template = task_template.create_prompt_template()

    return prompt_template


def initialize_dataset_and_loader(
    args: argparse.Namespace, 
    corpus: List[Dict], 
    full_to_subset_idx_map: Optional[Dict[int, int]], 
    retriever_search_results: List[Tuple[List[int], List[float]]], 
    tokenizer: PreTrainedTokenizer
) -> DataLoader:
    
    prompt_template = get_prompt_template(args)
    
    prompt_ds = PromptDataset(
        corpus=corpus, data_path=info[args.dataset]['test']['data_path'], 
        tokenizer=tokenizer, 
        max_tokenized_length=args.model_max_length - 2, 
        search_results=retriever_search_results,
        prompt_template=prompt_template,
        full_to_subset_idx_map=full_to_subset_idx_map,
        do_normalize_query=True, 
        num_documents_in_context=args.num_retrieved_documents,
        gold_position=args.gold_position, # None in these experiments
    )
        
    prompt_dataloader = DataLoader(
        prompt_ds,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=8,
        pin_memory=True,
    )
    return prompt_dataloader


def print_info(args: argparse.Namespace):
    print("INFO:")    
    print(f"DATA: {info[args.dataset]['test']['data_path']}")
    print(f"USE TEST: {args.use_test}")
    print(f"MODEL: {args.llm_id}")
    print(f"MODEL MAX LENGTH: {args.model_max_length}")
    print(f'MAX NEW TOKENS: {args.max_new_tokens}')
    print(f"USE MODEL CHAT TEMPLATE: {args.use_model_chat_template}")
    print(f"TASK WITH PROOF:", args.use_task_with_proof)
    print(f"GOLD POSITION: {args.gold_position}")
    print(f"NUM DOCUMENTS IN CONTEXT: {args.num_retrieved_documents}")
    print(f"BATCH SIZE: {args.batch_size}")
    print(f"SAVE EVERY: {args.save_every}")


def extract_generate_answers(
    args: argparse.Namespace, 
    generated_output: List[str]
) -> List[str]:
    answer_prefix = "Answer:"
    if args.use_model_chat_template:
        #answer_prefix = re.escape(chat_task_templates[args.llm_id]['answer_prefix'])

        answer_prefix = re.escape("Answer:") + r"\nmodel"

    print(f"Answer Prefix: {answer_prefix}")
    print(f"Generated Output: {generated_output}")

    generated_answers = []
    for output in generated_output:
        matches = list(re.finditer(answer_prefix, output))

        print(f"Have you find matches?: {matches}")

        match_idx = 0

        # When using the proof there is a one-shot example that already 
        # contains the string "Answer:". Thus, we should get the second (match_idx=1) match.
        if args.use_task_with_proof:
            match_idx = 1
            if args.use_model_chat_template and answer_prefix != "Answer:":
                match_idx = 0
 
        answer_end = matches[match_idx].end()
        response = output[answer_end:].strip()
        generated_answers.append(response)
    
    return generated_answers


def generate_and_save(
    args: argparse.Namespace, 
    llm: LLM, 
    prompt_dataloader: DataLoader
):
    # Info from arguments
    llm_id = args.llm_id
    num_doc = args.num_retrieved_documents
    save_every = args.save_every
    retriever_str = "contriever"
    padding_str = f"_{args.padding_strategy}{args.model_max_length}" if args.padding_strategy != "longest" else "" 
    chat_template_str = "_template" if args.use_model_chat_template else ""
    prompt_type = "retrieved_proof" if args.use_task_with_proof else "retrieved"

    # Create the saving directory
    llm_folder = llm_id.split("/")[1] if '/' in llm_id else llm_id
    saving_dir = f"{args.output_dir}/{args.dataset}/{llm_folder}/test/{prompt_type}/{retriever_str}/{num_doc}_doc"
    os.makedirs(saving_dir, exist_ok=True)

    all_info = []  
    for idx, prompt_batch in enumerate(tqdm(prompt_dataloader)):
        prompts = prompt_batch['prompt']
        generated_output = llm.generate(
            prompts,
            padding_strategy=args.padding_strategy, 
            max_new_tokens=args.max_new_tokens
        )

        generated_answers = extract_generate_answers(args, generated_output)
        prompt_batch['generated_answer'] = generated_answers
        all_info.append(prompt_batch)
        
        if (idx + 1) % save_every == 0 or (idx + 1) == len(prompt_dataloader):
            print(f"Saving at {idx + 1}...")
            file_name = f"{saving_dir}/numdoc{num_doc}_retr{args.num_retrieved_documents}{padding_str}{chat_template_str}_info_{idx+1}.pkl"
            write_pickle(all_info, file_name)
            all_info = []


def main():
    args = parse_arguments()

    print("Loading LLM...")
    llm_id = args.llm_id
    llm = LLM(
        llm_id, device, 
        quantization_bits=args.quantization_bits, 
        model_max_length=args.model_max_length
    )
    tokenizer = llm.tokenizer
    print("LLM loaded")


    print("Loading corpus and search results...")
    corpus, full_to_subset_idx_map = load_corpus(args)
    retriever_search_results = load_search_results(args)
    print("Corpus and search results loaded")


    print("Loading prompt dataset...")
    prompt_dataloader = initialize_dataset_and_loader(
        args, corpus, full_to_subset_idx_map, 
        retriever_search_results, tokenizer
    )
    print("Prompt dataset loaded")

    print_info(args)
    generate_and_save(args, llm, prompt_dataloader)



if __name__ == "__main__":
    seed_everything(SEED)
    main()

Loading LLM...


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.49s/it]


LLM loaded
Loading corpus and search results...
Corpus and search results loaded
Loading prompt dataset...
Prompt dataset loaded
INFO:
DATA: C:\Users\franc\Documents\Bridge_the_GAP\data\test_dataset.json
USE TEST: True
MODEL: google/gemma-2-2b-it
MODEL MAX LENGTH: 4096
MAX NEW TOKENS: 50
USE MODEL CHAT TEMPLATE: True
TASK WITH PROOF: False
GOLD POSITION: None
NUM DOCUMENTS IN CONTEXT: 5
BATCH SIZE: None
SAVE EVERY: 250


  0%|          | 1/2889 [01:04<51:54:01, 64.70s/it]

Answer Prefix: Answer:\nmodel
Generated Output: ['user\nYou are given a question and you must respond based on the provided documents. You must always provide an answer.\nDocuments:\nDocument [628506](Title: Nobel Prize in Physics) receive a diploma, a medal and a document confirming the prize amount. Nobel Prize in Physics The Nobel Prize in Physics () is a yearly award given by the Royal Swedish Academy of Sciences for those who have made the most outstanding contributions for mankind in the field of physics. It is one of the five Nobel Prizes established by the will of Alfred Nobel in 1895 and awarded since 1901; the others being the Nobel Prize in Chemistry, Nobel Prize in Literature, Nobel Peace Prize, and Nobel Prize in Physiology or Medicine. The first Nobel Prize in Physics was\nDocument [3546609](Title: E. C. George Sudarshan) had developed the breakthrough. In 2007, Sudarshan told the "Hindustan Times", "The 2005 Nobel prize for Physics was awarded for my work, but I wasn\'t 

...

Answer Prefix: Answer:\nmodel
Generated Output: ['user\nYou are given a question and you must respond based on the provided documents. You must always provide an answer.\nDocuments:\nDocument [2979957](Title: Revolution in Military Affairs) transformed warfare, and, as a result, the question is not one of "Does an RMA exist?" but, rather, "When did it begin, and what are its implications?" Tied to this are surprisingly persistent questions about the use and value of air power, now more accurately seen as aerospace power. If nothing else, given the record of precision air power application, aerospace power advocates should not still have to spend as much time as they do arguing the merits of three-dimensional war and precision attack\'s value to it. Modern joint service aerospace forces offer the most responsive, flexible, lethal, and devastating\nDocument [3526617](Title: Navy Marine Corps Intranet) perspective, produced results that were far from optimal.” NMCI consolidated roughly 6,

  0%|          | 5/2889 [01:18<7:08:11,  8.91s/it]

Answer Prefix: Answer:\nmodel
Generated Output: ['user\nYou are given a question and you must respond based on the provided documents. You must always provide an answer.\nDocuments:\nDocument [3932834](Title: Human Rights Day) and societies should "strive by progressive measures, national and international, to secure their universal and effective recognition and observance". The measure was received by both advocates and critics alike as "being more declarative than legislative, more suggestive than binding." Although the Declaration with its broad range of political, civil, economic, social and cultural rights is not a binding document, it inspired more than 60 human rights instruments which together constitute an international standard of human rights. Today the general consent of all United Nations Member States on the basic Human Rights laid down in the Declaration makes it even stronger\nDocument [182208](Title: Human rights) and peace in the world." The declaration was the first 

  0%|          | 5/2889 [01:24<13:35:39, 16.97s/it]


KeyboardInterrupt: 

In [ ]:
generated_output=['user\nYou are given a question and you must respond based on the provided documents. You must always provide an answer.\nDocuments:\nDocument [628506](Title: Nobel Prize in Physics) receive a diploma, a medal and a document confirming the prize amount. Nobel Prize in Physics The Nobel Prize in Physics () is a yearly award given by the Royal Swedish Academy of Sciences for those who have made the most outstanding contributions for mankind in the field of physics. It is one of the five Nobel Prizes established by the will of Alfred Nobel in 1895 and awarded since 1901; the others being the Nobel Prize in Chemistry, Nobel Prize in Literature, Nobel Peace Prize, and Nobel Prize in Physiology or Medicine. The first Nobel Prize in Physics was\nDocument [3546609](Title: E. C. George Sudarshan) had developed the breakthrough. In 2007, Sudarshan told the "Hindustan Times", "The 2005 Nobel prize for Physics was awarded for my work, but I wasn\'t the one to get it. Each one of the discoveries that this Nobel was given for work based on my research." Sudarshan also commented on not being selected for the 1979 Nobel, "Steven Weinberg, Sheldon Glashow and Abdus Salam built on work I had done as a 26-year-old student. If you give a prize for a building, shouldn’t the fellow who built the first floor be given the prize before those who built the second\nDocument [439756](Title: University of Chicago) Medal, which is rewarded annually to the best economist under the age of 40, has also been awarded to 4 current members of the university faculty. Notable faculty in physics have included the speed of light calculator A. A. Michelson, elementary charge calculator Robert A. Millikan, discoverer of the Compton Effect Arthur H. Compton, the creator of the first nuclear reactor Enrico Fermi, "the father of the hydrogen bomb" Edward Teller, "one of the most brilliant and productive experimental physicists of the twentieth century" Luis Walter Alvarez, Murray Gell-Mann who introduced the quark, second female Nobel laureate Maria Goeppert-Mayer, the\nDocument [1860765](Title: Tsung-Dao Lee) 1956, with her so-called Wu experiment. Lee was the youngest Nobel laureate after World War II until Malala Yousafzai was awarded the Nobel Peace Prize in 2014. He is the fourth youngest Nobel laureate in history after William L. Bragg (who won the prize at 25 with his father William H. Bragg in 1915), Werner Heisenberg (who won in 1932 also at 30) and Malala Yousafzai (awarded at just 17). Lee and Yang were the first Chinese laureates. Since he became a naturalized American citizen in 1962, Lee is also the youngest American ever to have won a Nobel Prize.\nDocument [2043329](Title: Universology) become the chief proponent of universology today. "Everything in this universe is part of an uninterrupted sequence of events" Mohri has said. In 1872 Andrews published "The Basic Outline of Universology" which was subtitled "An introduction to the newly discovered science of the universe, its elementary principles, and the first stages of their development in the special sciences." Ilya Romanovich Prigogine (born on January 25, 1917) was a Belgian and American physicist and chemist who was born in Russia and became a Nobel Prize laureate in chemistry. In the book "Order Out of Chaos: Man\'s New Dialogue With Nature", which\nQuestion: who got the first nobel prize in physics\nAnswer:\n\nmodel\nThe provided text does not state who received the first Nobel Prize in Physics.  It only states that the Nobel Prize in Physics was established in 1895 and awarded since 1901. \n']

answer_prefix="Answer:\n\nmodel\n"

for output in generated_output:
        print(output)
        matches = list(re.finditer(answer_prefix, output))
        print(matches)

user
You are given a question and you must respond based on the provided documents. You must always provide an answer.
Documents:
Document [628506](Title: Nobel Prize in Physics) receive a diploma, a medal and a document confirming the prize amount. Nobel Prize in Physics The Nobel Prize in Physics () is a yearly award given by the Royal Swedish Academy of Sciences for those who have made the most outstanding contributions for mankind in the field of physics. It is one of the five Nobel Prizes established by the will of Alfred Nobel in 1895 and awarded since 1901; the others being the Nobel Prize in Chemistry, Nobel Prize in Literature, Nobel Peace Prize, and Nobel Prize in Physiology or Medicine. The first Nobel Prize in Physics was
Document [3546609](Title: E. C. George Sudarshan) had developed the breakthrough. In 2007, Sudarshan told the "Hindustan Times", "The 2005 Nobel prize for Physics was awarded for my work, but I wasn't the one to get it. Each one of the discoveries that thi

In [ ]:
import torch
from transformers import (
    AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM
)
from typing import List, Tuple, Dict


class BridgeModel:
    """
    Bridge Model for selecting and ranking documents by generating document IDs.
    """
    def __init__(
        self, 
        model_id: str, 
        device: str = 'cuda', 
        model_max_length: int = 512
    ):
        self.device = device
        self.model_max_length = model_max_length

        # Initialize the seq2seq model and tokenizer
        self.model, self.tokenizer = self._initialize_model_tokenizer(model_id)

    def _initialize_model_tokenizer(self, model_id: str) -> Tuple[AutoModelForSeq2SeqLM, AutoTokenizer]:
        """
        Initializes the seq2seq model and tokenizer with the given model ID.
        """
        model_config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
        model_config.max_seq_len = self.model_max_length

        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            config=model_config,
            torch_dtype=torch.bfloat16,
            device_map="auto",
        ).to(self.device)
        model.eval()  # Set the model to evaluation mode

        tokenizer = AutoTokenizer.from_pretrained(
            model_id, 
            model_max_length=self.model_max_length,
            padding_side="left",
            truncation_side="left"
        )
        tokenizer.pad_token = tokenizer.eos_token  # Set pad token

        return model, tokenizer

    def generate(
        self, 
        prompt: str, 
        max_new_tokens: int = 15
    ) -> List[str]:
        """
        Generates the ordered document IDs based on the query and documents.
        """
        # Tokenize input
        inputs = self.tokenizer(
            prompt, 
            return_tensors="pt", 
            max_length=self.model_max_length, 
            padding=True, 
            truncation=True
        ).to(self.device)

        # Generate output
        generated_ids = self.model.generate(
            **inputs,
            do_sample=False,  # Deterministic output
            max_new_tokens=max_new_tokens,
            repetition_penalty=1.1,
            pad_token_id=self.tokenizer.pad_token_id,
            eos_token_id=self.tokenizer.eos_token_id,
        )

        # Decode output
        return self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].split()

# Example Usage
if __name__ == "__main__":
    model_id = "t5-small"  # Replace with a seq2seq model like T5 or BART
    bridge = BridgeModel(model_id=model_id)

    output = bridge.generate(prompt)
    print("Output document IDs:", output)